<a href="https://colab.research.google.com/github/MarconeRafael/Analise_visual_dados/blob/main/Estrategista_neg%C3%B3cios_vision%C3%A1rio_yourclub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Raspagemm e tratamento de dados

In [15]:
!pip install -q requests beautifulsoup4 unidecode
!pip install -q PyMuPDF


In [16]:
import os
import re
import csv
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
from sklearn.model_selection import train_test_split
import fitz  # PyMuPDF para trabalhar com PDFs

# Função para baixar arquivo do Google Drive
def download_pdf_from_drive(file_id, save_path):
    base_url = "https://drive.google.com/uc?export=download&id="
    url = f"{base_url}{file_id}"
    try:
        response = requests.get(url, stream=True)
        with open(save_path, "wb") as f:
            f.write(response.content)
        print(f"PDF salvo em: {save_path}")
    except Exception as e:
        print(f"Erro ao baixar o arquivo {file_id}: {e}")

# Diretório para salvar os dados raspados
DATA_DIR = "empreendedorismo_dados"
os.makedirs(DATA_DIR, exist_ok=True)

# Fontes para raspagem e PDFs
URLS = [
    "https://rockcontent.com/br/blog/o-que-e-growth-marketing/",
    "https://br.hubspot.com/blog/marketing/growth-marketing",
    "https://www.ekyte.com/guide/pt-br/conceitos/o-que-e-growth-marketing-e-como-aplicar-essa-metodologia-na-pratica"


]
# IDs dos arquivos do Google Drive (extraídas das URLs compartilhadas)
PDF_IDS = [
    "1vtTTcyX9UBnvycDKbRPQjVjwEo-K_Py7",
    "1eRSPyW0j4410sGYiz8CPdv758PvJX4qN",
    "1tNMnDv6FKBz95KLTkiU2UNCBtLyQsA15",
    "1iYq_4RYkG_En4NLUEHJw5atb9kwFnx5n",
    "1LZcoXtZf0UXlTNK08S2mNpCxLFH7IAcS",
    "1Jp5PXCSQhPNZq6fulSh-5N9rVB55zLSq",
    "1ufTR2-nkY83ROON8ka6YxdgqJdhC2maK"
]
# Função para extrair texto de PDFs
def extract_text_from_pdf(pdf_path):
    print(f"Extraindo texto de: {pdf_path}")
    try:
        doc = fitz.open(pdf_path)  # Abre o PDF
        text = ""
        for page_num in range(doc.page_count):  # Percorre todas as páginas
            page = doc.load_page(page_num)
            text += page.get_text()  # Extrai texto da página
        return text
    except Exception as e:
        print(f"Erro ao extrair de {pdf_path}: {e}")
        return ""

# Função para raspar conteúdo de uma URL
def scrape_url(url):
    print(f"Raspando dados de: {url}")
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        # Busca conteúdo mais específico
        content = soup.find("main") or soup.find("body") or soup
        paragraphs = content.find_all("p")
        text = "\n".join([p.get_text() for p in paragraphs])
        return text
    except Exception as e:
        print(f"Erro ao raspar {url}: {e}")
        return ""

# Coleta de dados
def collect_data(urls, pdf_ids):
    all_text = []

    # Raspando dados das URLs
    for url in urls:
        text = scrape_url(url)
        if text:
            all_text.append(text)

    # Baixando e extraindo dados dos PDFs
    for pdf_id in pdf_ids:
        save_path = os.path.join(DATA_DIR, f"livro_{pdf_id}.pdf")
        download_pdf_from_drive(pdf_id, save_path)
        text = extract_text_from_pdf(save_path)
        if text:
            all_text.append(text)

    return " ".join(all_text)

# Limpeza do texto
def clean_text(text):
    text = re.sub(r"<[^>]+>", "", text)  # Remove tags HTML
    text = unidecode(text)  # Remove acentos
    text = re.sub(r"\s+", " ", text)  # Normaliza espaços
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9áéíóúàãõç\s.,!?]", "", text)  # Mantém pontuação básica
    return text.strip()

# Divisão dos dados
def split_data(data, train_ratio=0.8, val_ratio=0.1):
    train, temp = train_test_split(data, test_size=1 - train_ratio, random_state=42)
    val, test = train_test_split(temp, test_size=val_ratio / (1 - train_ratio), random_state=42)
    return train, val, test

# Salvar dados em arquivos CSV
def save_data_to_csv(data, filename):
    filepath = os.path.join(DATA_DIR, filename)
    with open(filepath, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["sentences"])
        for sentence in data:
            writer.writerow([sentence])

# Pipeline principal
def main():
    print("Iniciando coleta de dados...")
    raw_data = collect_data(URLS, PDF_IDS)
    print(f"Dados coletados ({len(raw_data)} caracteres).")

    print("Limpando dados...")
    cleaned_data = clean_text(raw_data)
    print(f"Texto limpo: {cleaned_data[:500]}")  # Exibe parte do texto limpo para análise

    print("Dividindo em sentenças...")
    # Dividindo o texto em sentenças, separando por pontuação final (., ?, !)
    sentences = re.split(r'(?<=[.!?]) +', cleaned_data)
    sentences = [sentence.strip() for sentence in sentences if len(sentence.strip()) > 10]

    print(f"Dados limpos ({len(sentences)} sentenças).")

    if len(sentences) < 50:
        print("Poucas sentenças disponíveis. Adicione mais fontes ou refine o processamento.")
        return

    print("Dividindo os dados...")
    train, val, test = split_data(sentences)
    print(f"Dados divididos: {len(train)} treino, {len(val)} validação, {len(test)} teste.")

    print("Salvando dados em arquivos CSV...")
    save_data_to_csv(train, "treinamento.csv")
    save_data_to_csv(val, "validacao.csv")
    save_data_to_csv(test, "teste.csv")
    print(f"Dados salvos em {DATA_DIR}.")

if __name__ == "__main__":
    main()


Iniciando coleta de dados...
Raspando dados de: https://rockcontent.com/br/blog/o-que-e-growth-marketing/
Raspando dados de: https://br.hubspot.com/blog/marketing/growth-marketing
Raspando dados de: https://www.ekyte.com/guide/pt-br/conceitos/o-que-e-growth-marketing-e-como-aplicar-essa-metodologia-na-pratica
Erro ao raspar https://www.ekyte.com/guide/pt-br/conceitos/o-que-e-growth-marketing-e-como-aplicar-essa-metodologia-na-pratica: 403 Client Error: Forbidden for url: https://www.ekyte.com/guide/pt-br/conceitos/o-que-e-growth-marketing-e-como-aplicar-essa-metodologia-na-pratica
PDF salvo em: empreendedorismo_dados/livro_1vtTTcyX9UBnvycDKbRPQjVjwEo-K_Py7.pdf
Extraindo texto de: empreendedorismo_dados/livro_1vtTTcyX9UBnvycDKbRPQjVjwEo-K_Py7.pdf
PDF salvo em: empreendedorismo_dados/livro_1eRSPyW0j4410sGYiz8CPdv758PvJX4qN.pdf
Extraindo texto de: empreendedorismo_dados/livro_1eRSPyW0j4410sGYiz8CPdv758PvJX4qN.pdf
PDF salvo em: empreendedorismo_dados/livro_1tNMnDv6FKBz95KLTkiU2UNCBtLyQsA15

#Treinando nosso modelo de empreendedorismo

In [17]:
!pip install -q transformers torch datasets


In [21]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [19]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import pandas as pd

# Carregar o tokenizer do BERTimbau
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Carregar o modelo BERTimbau pré-treinado
model = BertForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Carregar os dados de treinamento
train_df = pd.read_csv("empreendedorismo_dados/treinamento.csv")
val_df = pd.read_csv("empreendedorismo_dados/validacao.csv")
test_df = pd.read_csv("empreendedorismo_dados/teste.csv")

# Função para tokenizar as sentenças
# Função para tokenizar as sentenças com truncamento para 512 tokens
def tokenize_function(examples):
    return tokenizer(examples['sentences'], padding="max_length", truncation=True, max_length=512)

# Preparar o dataset
train_dataset = Dataset.from_pandas(train_df[['sentences']])
val_dataset = Dataset.from_pandas(val_df[['sentences']])

# Tokenizar os dados
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)



# Criar um data collator para Masked Language Modeling (MLM)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='./especialista_empreendedorismo',  # Onde salvar o modelo treinado
    evaluation_strategy="epoch",  # Avaliação a cada época
    learning_rate=5e-5,  # Taxa de aprendizado
    per_device_train_batch_size=8,  # Tamanho do batch para treinamento
    per_device_eval_batch_size=8,  # Tamanho do batch para avaliação
    num_train_epochs=3,  # Número de épocas
    weight_decay=0.01,  # Decaimento do peso
)

# Usar o Trainer da Hugging Face para o treinamento
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,  # Usar o collator para MLM
)

# Treinar o modelo
trainer.train()

# Salvar o modelo treinado
model.save_pretrained('./model/especialista_empreendedorismo')
tokenizer.save_pretrained('./model/especialista_empreendedorismo')

print("Modelo treinado e salvo como 'especialista_empreendedorismo'.")


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/8908 [00:00<?, ? examples/s]

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,2.404800,2.069829
2,2.086400,1.880966
3,1.864100,1.812348


Modelo treinado e salvo como 'especialista_empreendedorismo'.


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Carregar o tokenizer e o modelo treinado
tokenizer = BertTokenizer.from_pretrained('./model/especialista_empreendedorismo')
model = BertForMaskedLM.from_pretrained('./model/especialista_empreendedorismo')

# Função para conversar com o modelo
def chat_with_model(input_text):
    # Codificar a entrada do usuário, adicionando o token de separação [SEP]
    new_user_input_ids = tokenizer.encode(input_text + tokenizer.sep_token, return_tensors="pt")

    # Gerar uma resposta do modelo com ajustes para evitar repetições
    bot_output = model.generate(
        new_user_input_ids,
        max_length=50,  # Limitar o tamanho máximo da resposta
        pad_token_id=tokenizer.pad_token_id,  # Definir o token de padding
        no_repeat_ngram_size=2,  # Impedir repetições de bigramas (2 palavras)
        top_k=10,  # Reduzir o número de opções mais prováveis
        top_p=0.85,  # Limitar a amostragem top-p
        temperature=0.7,  # Aumenta a criatividade da resposta
        do_sample=True,  # Habilita amostragem
        num_return_sequences=1  # Retorna uma única resposta
    )

    # Decodificar a resposta gerada
    bot_output_text = tokenizer.decode(bot_output[0], skip_special_tokens=True)

    return bot_output_text



# Testando a interação

while True:
  user_input = input(": ")
  response = chat_with_model(user_input)
  print(f"Bot: {response}")
  if response == "0":
    break

: oi
Bot: oi.. 1. 2. 3. 6. 7. 5. 9. 10. xiiiii.ivi xvii oii. oviii xiix xvii lvix
: bom dia
Bot: bom dia.. 2. 0. 1. 7. 3 Muito obrigado.!!.??!?. xxxiii xviiiii xii. oi. ix. uiixvix
